In [ ]:
from flask import Flask, request, jsonify, render_template
from elasticsearch import Elasticsearch, ConnectionError
from sentence_transformers import SentenceTransformer
import re

# Initialize Flask app
app = Flask(__name__)

# Initialize Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Load the model for embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def generate_embedding(text):
    """Generates an embedding for the given text."""
    return model.encode(text).tolist()

def highlight_text(text, terms, color="orange"):
    """Highlights multiple terms in the text with HTML, ignoring case."""
    for term in terms:
        regex = re.compile(re.escape(term), re.IGNORECASE)
        text = regex.sub(lambda match: f'<span style="background-color: {color};">{match.group(0)}</span>', text)
    return text

def realizar_busqueda(user_query, start_date, end_date, search_type):
    terms = user_query.split()
    
    # Generate the embedding for the query
    query_embedding = generate_embedding(user_query)

    # Determine the Elasticsearch index based on search_type
    index_name = "boja_index" if search_type == "BOJA" else "boe_index"

    # Function to perform searches in different fields
    def search_in_field(field, query, start_date, end_date, size=10):
        must_conditions = [
            {"multi_match": {"query": query, "fields": [field], "operator": "and"}}
        ]
        
        if start_date and end_date:
            must_conditions.append({"range": {"d_date": {"gte": start_date, "lte": end_date}}})
        
        try:
            response = es.search(index=index_name, body={
                "query": {
                    "bool": {
                        "must": must_conditions
                    }
                },
                "size": size
            })
            return response['hits']['hits']
        except ConnectionError as e:
            print(f"Connection error: {e}")
            return []

    # Perform searches in each specified field
    results_sumario = search_in_field("t_asumarioNoHtml", user_query, start_date, end_date, 10)
    results_cuerpo = search_in_field("t_bodyNoHtml", user_query, start_date, end_date, 10)
    results_entities_asumario = search_in_field("entities_asumario", user_query, start_date, end_date, 5)
    results_entities_body = search_in_field("entities_body", user_query, start_date, end_date, 5)

    # Perform the search using embeddings with date range if provided
    if start_date and end_date:
        try:
            response_embeddings = es.search(index=index_name, body={
                "query": {
                    "bool": {
                        "must": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                    "params": {"query_vector": query_embedding}
                                }
                            }
                        },
                        "filter": {
                            "range": {"d_date": {"gte": start_date, "lte": end_date}}
                        }
                    }
                },
                "size": 10
            })
            results_embeddings = response_embeddings['hits']['hits']
        except ConnectionError as e:
            print(f"Connection error: {e}")
            results_embeddings = []
    else:
        try:
            response_embeddings = es.search(index=index_name, body={
                "query": {
                    "script_score": {
                        "query": {"match_all": {}},
                        "script": {
                            "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                            "params": {"query_vector": query_embedding}
                        }
                    }
                },
                "size": 10
            })
            results_embeddings = response_embeddings['hits']['hits']
        except ConnectionError as e:
            print(f"Connection error: {e}")
            results_embeddings = []

    # Combine the results and remove duplicates
    combined_results = {hit['_id']: hit for hit in (results_sumario + results_cuerpo + results_entities_asumario + results_entities_body + results_embeddings)}
    combined_results = list(combined_results.values())[:30]  # Get the top 30 results

    # Create the HTML output and apply highlighting
    results = []
    for hit in combined_results:
        sumario = hit['_source'].get('t_asumarioNoHtml', '')
        cuerpo = hit['_source'].get('t_bodyNoHtml', '')

        # Skip results without sumario and cuerpo
        if not sumario and not cuerpo:
            continue

        url = hit['_source']['t_publicUrl']
        fecha = hit['_source']['d_date']
        sumario = highlight_text(sumario, terms)
        cuerpo = highlight_text(cuerpo, terms)
        
        result = {
            "ID": hit['_id'],
            "URL": url,
            "Fecha": fecha,
            "Sumario": sumario,
            "Cuerpo": cuerpo
        }
        results.append(result)

    return results

@app.route('/search', methods=['POST'])
def search():
    data = request.json  # Get the JSON data from the request body
    print(data)  # Print the data to the console
    keyword = data.get('query', '')
    start_date = data.get('startDate', '')
    end_date = data.get('endDate', '')
    sort_option = data.get('sortBy', 'relevance')
    search_type = data.get('searchType', 'BOJA')  # Get the search type, default to 'BOJA'
   
    results = realizar_busqueda(keyword, start_date, end_date, search_type)
    max_length = 500  # You can adjust this value as needed
    for result in results:
        if len(result['Sumario']) > max_length:
            result['Sumario'] = result['Sumario'][:max_length] + '...'
        if len(result['Cuerpo']) > max_length:
            result['Cuerpo'] = result['Cuerpo'][:max_length] + '...'
   
    print(results)
    return jsonify(result=results)  # Return the results as JSON

@app.route('/', methods=['GET'])
def index():
    return render_template('BOJA.ia.html')

if __name__ == '__main__':
    app.run(debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2024 05:28:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 05:28:55] "GET /static/buscar.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2024 05:28:55] "GET /static/ayuda7.jpg HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2024 05:28:55] "GET /static/favicon.ico HTTP/1.1" 304 -


{'query': 'universidad huelva', 'startDate': '', 'endDate': '', 'sortBy': 'relevance', 'searchType': 'BOJA'}


127.0.0.1 - - [03/Jun/2024 05:29:00] "POST /search HTTP/1.1" 200 -


[{'ID': 'disposiciones_boja-disposition.2019.116.40', 'URL': 'https://www.juntadeandalucia.es/eboja/2019/116/BOJA19-116-00006-9301-01_00157637.pdf', 'Fecha': '2019-06-19', 'Sumario': 'Resolución de 5 de junio de 2019, de la <span style="background-color: orange;">Universidad</span> de <span style="background-color: orange;">Huelva</span>, por la que se publica el Reglamento de Tablón Electrónico Oficial de la <span style="background-color: orange;">Universidad</span> de <span style="background-color: orange;">Huelva</span> y Boletín Oficial de la <span style="background-color: orange;">Universidad</span> de <span style="background-color: orange;">Huelva</span>, aprobado en Co...', 'Cuerpo': 'El Consejo de Gobierno de la <span style="background-color: orange;">Universidad</span> de <span style="background-color: orange;">Huelva</span> aprobó con fecha de 4 de mayo de 2018 el Reglamento de Creación y Funcionamiento de la Sede Electrónica, publicado en el Boletín Oficial de la Junta de An

...

{'query': 'décret sur les droits de succession dans gouvernement andalou', 'startDate': '', 'endDate': '', 'sortBy': 'relevance', 'searchType': 'BOJA'}
[{'ID': 'disposiciones_boja-disposition.1986.40.1', 'URL': 'https://www.juntadeandalucia.es/boja/1986/40/d1.pdf', 'Fecha': '1986-05-09', 'Sumario': 'LEY 4/1986, <span style="background-color: orange;">de</span> 5 <span style="background-color: orange;">de</span> mayo, <span style="background-color: orange;">de</span>l Patrimonio <span style="background-color: orange;">de</span> la Comunidad Autónoma <span style="background-color: orange;">de</span> Andalucía.', 'Cuerpo': 'Ley 4/1985, <span style="background-color: orange;">de</span> 5 <span style="background-color: orange;">de</span> mayo, <span style="background-color: orange;">de</span>l Patrimonio <span style="background-color: orange;">de</span> la Comunidad Autónoma <span style="background-color: orange;">de</span> Andalucía. EL PRESI<span style="background-color: orange;">DE</span

127.0.0.1 - - [03/Jun/2024 19:34:55] "POST /search HTTP/1.1" 200 -


{'query': 'décret sur les droits de succession dans gouvernement andalou', 'startDate': '', 'endDate': '', 'sortBy': 'relevance', 'searchType': 'BOE'}
[{'ID': 'disposiciones_boe-BOE-A-2023-16527', 'URL': 'https://boe.es/boe/dias/2023/07/17/pdfs/BOE-A-2023-16527.pdf', 'Fecha': '2023-07-17', 'Sumario': 'Resolución <span style="background-color: orange;">de</span> 29 <span style="background-color: orange;">de</span> junio <span style="background-color: orange;">de</span> 2023, <span style="background-color: orange;">de</span> la Dirección General <span style="background-color: orange;">de</span> Seguridad Jurídica y Fe Pública, en el recurso interpuesto contra la negativa <span style="background-color: orange;">de</span> la registradora <span style="background-color: orange;">de</span> la propie...', 'Cuerpo': 'Resolución <span style="background-color: orange;">de</span> 29 <span style="background-color: orange;">de</span> junio <span style="background-color: orange;">de</span> 2023, <spa

127.0.0.1 - - [03/Jun/2024 19:47:52] "POST /search HTTP/1.1" 200 -


{'query': 'ley 10/2018', 'startDate': '', 'endDate': '', 'sortBy': 'relevance', 'searchType': 'BOE'}
[{'ID': 'disposiciones_boe-BOE-A-2023-18413', 'URL': 'https://boe.es/boe/dias/2023/08/15/pdfs/BOE-A-2023-18413.pdf', 'Fecha': '2023-08-15', 'Sumario': '<span style="background-color: orange;">Ley</span> 10/2023, de 7 de agosto, de derogación de la <span style="background-color: orange;">Ley</span> 6/2018, de 26 de noviembre, de protección de los animales en la Comunidad Autónoma de La Rioja.', 'Cuerpo': '<span style="background-color: orange;">Ley</span> 10/2023, de 7 de agosto, de derogación de la <span style="background-color: orange;">Ley</span> 6/2018, de 26 de noviembre, de protección de los animales en la Comunidad Autónoma de La Rioja.'}, {'ID': 'disposiciones_boe-BOE-A-2023-13956', 'URL': 'https://boe.es/boe/dias/2023/06/12/pdfs/BOE-A-2023-13956.pdf', 'Fecha': '2023-06-12', 'Sumario': 'Pleno. Sentencia 45/2023, de 10 de mayo de 2023. Recurso de inconstitucionalidad 4129-2018. In